## Imports & Directories

In [ ]:
#-- Packages --#

#--- Operational ---#
import os
import sys 
import pandas as pd
import numpy as np
import geopandas as gpd
from pathlib import Path
import json
import re
import geopandas as gpd

#--- Visualisations ---#
import plotly.express as px
import plotly.graph_objects as go

#-- Directories --#
nb_dir = Path.cwd()
REPO_ROOT = nb_dir.parent
data_dir = REPO_ROOT / 'data/'
processed_dir = data_dir / 'processed/'
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

In [ ]:
#-- Helper Functions --#
def extract_max_year(path):
    """
    Identify shapefile with latest year of available data
    """
    years = re.findall(r"\d{4}", path.stem)
    return max(map(int, years)) if years else -1

In [ ]:
#-- Files --#

# Avalanche Canada polygons (GeoJSON)
avcan_path = REPO_ROOT / "data/external/avalanche_canada/canadian_subregions.geojson"
avcan = gpd.read_file(avcan_path)

# NBAC / BC fire perimeters 

fires_dir = REPO_ROOT / "data/processed/Canada_fires"
shp_files = list(fires_dir.glob("*.shp"))

if not shp_files:
    raise FileNotFoundError(f"No shapefiles found in {fires_dir}\n")

fires_path = max(shp_files, key=extract_max_year)

print(f"Loading latest Canada fires shapefile... \n File name: {fires_path.name}")
AvCan_fires = gpd.read_file(fires_path)
print(f"National Canada Fires loaded. {AvCan_fires.crs}")
# Province / Territories boundaries

provinces = gpd.read_file(REPO_ROOT / "data/external/stats_canada/boundaries/lpr_000b21a_e.shp")

print(f"\nAvalanche Canada Regions loaded. {avcan.crs}")
print(f"Canadian Province / Territory boundaries loaded. {provinces.crs}")

## Aggregations

In [ ]:
# AvCanada Ski Regions
print('Isolate all AvCanada Ski regions')
regions = avcan[["reference_region","polygon_name", "geometry"]]   # adjust column names as needed


In [ ]:
regions.head(10)

In [ ]:
print("Attach Province / Territory to all AvCanada Ski Regions")

# Make sure both layers use the same CRS
provinces = provinces.to_crs(regions.crs)
print(f'Pronvince CRS type change: {provinces.crs}')

regions_centroids = regions.copy()

# Extract AvCan region identifed by centroid valye
regions_centroids["geometry"] = regions_centroids.geometry.centroid

print('Join AvCan region to province by shapefile bounadies over centriods')
# Join AvCan region to province by shapefile boundaries across centroids
regions_with_admin = gpd.sjoin(
    regions_centroids,
    provinces[["PRENAME", "geometry"]],   # English name
    how="left",
    predicate="within"
).drop(columns="index_right")


regions_with_admin = regions_with_admin.rename(columns={"PRENAME": "admin_area"})

In [ ]:
regions_with_admin.value_counts('admin_area')

In [ ]:
regions_with_admin = regions_with_admin.drop(columns="geometry")
regions_with_admin = regions.merge(
    regions_with_admin[["reference_region", "polygon_name", "admin_area"]],
    on=["reference_region", "polygon_name"],
    how="left"
)

In [ ]:
regions_with_admin.head(10)

In [ ]:
print('Overlay all AvCan Fires into respective AvCanada Ski Regions')
AvCan_fires=AvCan_fires.drop(columns="admin_area")
# Overlay BcFires to respective AvCanada Ski Regions
fires_regions = gpd.overlay(
    AvCan_fires,
    regions_with_admin,
    how="intersection"   # intersection of fire and region polygons. Cutting fires by region borders
)


In [ ]:
# Make sure we use a projected CRS in metres for area calc
if fires_regions.crs.is_geographic:  # e.g. EPSG:4326
    fires_regions = fires_regions.to_crs("EPSG:3978")  # Canada Lambert as an example

print(f'''Convert fires_region variable CRS type to projected coordinates.
    Projected CRS type: {fires_regions.crs} (metres)
    Projected CRS name: {fires_regions.crs.name}
''')



In [ ]:
fires_regions['admin_area'].value_counts()

In [ ]:
print(f'Create dataframe with fire statistics for each AvCanada Ski Region')

# Create hectare area for each AvCanada Ski Region
fires_regions["area_ha"] = fires_regions.geometry.area / 10_000

# fires_regions has: gid, year, adj_ha, reference_region, polygon_name, area_ha, geometry, ...

fire_stats = (
    fires_regions
    .groupby(["admin_area","reference_region", "polygon_name"])
    .agg(
        n_fires=("gid", "nunique"),     # GID = year + fire id, unique per fire event
        burned_ha=("area_ha", "sum")    # area_ha = clipped fire area in that region
    )
    .reset_index()
)



In [ ]:
fire_stats

In [ ]:
# 2) One row per AvCan polygon, with geometry + admin_area
region_stats = regions_with_admin[["reference_region", "polygon_name", "admin_area", "geometry"]].drop_duplicates()

In [ ]:
# Merge all regions with fire statistics
region_stats = region_stats.merge(
    fire_stats,
    on=["admin_area","reference_region", "polygon_name"],
    how="left"        # keep ALL regions, even if no match in fire_stats
)

# Replace NaN with 0 for regions without fires
region_stats[["n_fires", "burned_ha"]] = region_stats[["n_fires", "burned_ha"]].fillna(0)


In [ ]:


# 1. Convert AvCan GeoDataFrame to a GeoJSON dict
avcan_geojson = json.loads(avcan.to_json())


year_min = fires_regions["year"].min()
year_max = fires_regions["year"].max()

title_text = (
    "Number of Fires in Avalanche Canada Regions"
    f"<br><span style='font-size:12px; color:gray;'>Between years: {year_min}–{year_max}</span>"
)

# 2. Build choropleth
fig = px.choropleth_map(
    data_frame=region_stats,
    geojson=avcan_geojson,
    locations="polygon_name",                 # column in region_stats
    featureidkey="properties.polygon_name",   # matching property in GeoJSON
    color="n_fires",
    color_continuous_scale="YlOrRd",
    range_color=(0, region_stats["n_fires"].max()),
    map_style="outdoors",           # or "open-street-map" if no token
    zoom=4,
    center={"lat": 54, "lon": -123},         # rough center on BC
    opacity=0.7,
        labels={
        "polygon_name": "AvCan region",
        "admin_area": "Province",
        "n_fires": "Number of fires",
    },
    hover_name="polygon_name",                       # big bold line
    hover_data={
        "admin_area": True,      # will use label "Province"
        "n_fires": True,         # will use label "Number of fires"
        "burned_ha": ':.0f',     # optional formatting example
    },
    title=title_text
)


fig.update_layout(
    margin={"r":0, "t":50, "l":0, "b":0},
    title_x=0.5,  # center the title,
    autosize=True
)
fig.show()


In [ ]:
fires_regions.head(10)

In [ ]:
region_stats

In [ ]:
region_stats.head(10)

In [ ]:
# fires_regions has: YEAR, reference_region, polygon_name, GID (or NFIREID), area_ha, ...

fires_by_year_region_raw = (
    fires_regions
    .groupby(["year", "admin_area","reference_region", "polygon_name"])
    .agg(
        n_fires=("gid", "nunique"),     # number of distinct fires hitting that region in that year
        burned_ha=("area_ha", "sum")   # optional, total burned area in that region-year
    )
    .reset_index()
)


In [ ]:

# All years you care about (option A: from data)
years = sorted(fires_regions["year"].unique())

# Or, if you want to force 2018–2024 even if one is missing:
# years = list(range(2018, 2025))

years_df = pd.DataFrame({"year": years})

# All AvCan regions (use the same `regions` you used before)
region_keys = regions_with_admin[["admin_area","reference_region", "polygon_name"]].drop_duplicates()

# Cartesian product (cross join) of years × regions
full_grid = (
    years_df.assign(_key=1)
    .merge(region_keys.assign(_key=1), on="_key")
    .drop(columns="_key")
)


In [ ]:
fires_by_year_region = (
    full_grid
    .merge(
        fires_by_year_region_raw,
        on=["year", "admin_area","reference_region", "polygon_name"],
        how="left"
    )
)

# Replace NaNs (no fires) with 0
fires_by_year_region["n_fires"] = fires_by_year_region["n_fires"].fillna(0).astype(int)
fires_by_year_region["burned_ha"] = fires_by_year_region["burned_ha"].fillna(0.0)


In [ ]:
fires_by_year_region.sort_values(by='n_fires', ascending=True)

In [ ]:
title_text = (
    "Number of Fires in Avalanche Canada Regions Per Year"
)

fig1 = px.choropleth_map(
    data_frame=fires_by_year_region,
    geojson=avcan_geojson,
    locations="polygon_name",                 # column in fires_by_year_region
    featureidkey="properties.polygon_name",   # matching field in GeoJSON
    color="n_fires",
    animation_frame="year",                   # <-- slider by year
    color_continuous_scale="YlOrRd",
    range_color=(0, fires_by_year_region["n_fires"].max()),
    map_style="outdoors",
    center={"lat": 54, "lon": -123},
    zoom=4,
    opacity=0.7,
        labels={
        "polygon_name": "AvCan region",
        "admin_area": "Province",
        "n_fires": "Number of fires",
    },
    hover_name="polygon_name",                       # big bold line
    hover_data={
        "admin_area": True,      # will use label "Province"
        "n_fires": True,         # will use label "Number of fires"
        "burned_ha": ':.0f',     # optional formatting example
    },
    title=title_text
)

fig1.update_layout(
    margin={"r":0, "t":40, "l":0, "b":0},
    title_x=0.5,  # center the title,
    autosize=True)
fig1.show()


In [ ]:
import geopandas as gpd

# Ensure everything is in lat/lon (EPSG:4326) for web maps
avcan_wgs = avcan.to_crs(4326)
fires_wgs = fires_regions.to_crs(4326).copy()

# Fire centroids for plotting as points
fires_wgs["lon"] = fires_wgs.geometry.centroid.x
fires_wgs["lat"] = fires_wgs.geometry.centroid.y


In [ ]:
# Minimal df with one row per polygon
regions_for_plot = avcan_wgs.copy()
regions_for_plot["value"] = 1  # dummy

avcan_geojson = json.loads(avcan_wgs.to_json())

fig = px.choropleth_map(
    data_frame=regions_for_plot,
    geojson=avcan_geojson,
    locations="polygon_name",
    featureidkey="properties.polygon_name",
    color="value",
    color_continuous_scale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0)"]],  # transparent fill
    map_style="outdoors",
    zoom=5,
    center={"lat": 54, "lon": -123},
    opacity=0.4,
)

# Get rid of colorbar from dummy 'value'
fig.update_coloraxes(showscale=False)




In [ ]:
fires_wgs

In [ ]:
# For animation, we only need the fire points df
fig_fires = px.scatter_map(
    fires_wgs,
    lat="lat",
    lon="lon",
    color="adj_ha",
    animation_frame="year",
    map_style="carto-positron",
    center={"lat": 54, "lon": -123},
    zoom=5,
    labels={
        "gid":"Fire ID",
        "polygon_name": "AvCan region",
        "admin_area": "Province",
        "n_fires": "Number of fires",
    },
    hover_name="gid",
    hover_data={
        "gid": True,
        "admin_area": True,
        "reference_region": True,
        "polygon_name": True,
        "adj_ha": ':.0f',
    }
)

# Add AvCan polygons as a GeoJSON layer under the points
fig_fires.update_layout(
    map=dict(
        style="carto-positron",
        center={"lat": 54, "lon": -123},
        zoom=5,
        layers=[
            dict(
                source=avcan_geojson,
                type="fill",
                below="traces",
                color="rgba(0,0,0,0.05)",   # very light tint so it doesn’t overpower fires
            )
        ],
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

fig_fires.show()


In [ ]:
AvCan_fires.head(10)